In [2]:
from efficient_kan import KAN
import torch
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.nn.functional as F
from matplotlib import pyplot as plt

In [5]:
# Original model
# model = KAN([16,4,5], grid_size=40, spline_order=4, grid_eps=0.03, base_activation=nn.GELU, grid_range=[-8,8])
!ls

convert_kan_to_cpp.py  templates  test.ipynb


In [6]:
SAVE_PATH = "../mini_kan.pt"
model = torch.load(SAVE_PATH, weights_only=False)

In [7]:
def get_activation_values(layer_i, inp_node, out_node, grid_range=[-8, 8], resolution=256):

    layer = model.layers[layer_i]

    # Create dummy input
    array = np.linspace(grid_range[0], grid_range[1], resolution)
    stacked_array = np.hstack([[array]*layer.in_features]).T
    x = torch.from_numpy(stacked_array).float().to(device)

    #Loop through each activation function
    base_output = layer.base_activation(x)[: , inp_node] * layer.base_weight[out_node, inp_node]
    spline_output = F.linear(layer.b_splines(x)[:, out_node, :], layer.scaled_spline_weight[out_node, inp_node, :])

    return base_output + spline_output


In [8]:
for i, layer in enumerate(model.layers):
    print(f"Layer {i} has {layer.in_features} input nodes and {layer.out_features} output nodes")

Layer 0 has 16 input nodes and 4 output nodes
Layer 1 has 4 input nodes and 5 output nodes


In [9]:
get_activation_values(1, 0, 4)

IndexError: index 4 is out of bounds for dimension 1 with size 4